In [1]:
import json

with open('train-v2.0.json', 'r') as f_train:
    train = json.load(f_train)
    train_0 = train['data'][0]['paragraphs'][:10]

with open('dev-v2.0.json', 'r') as f_test:
    test = json.load(f_test)
    test_0 = test['data'][0]['paragraphs'][:10]

In [2]:
import logging

from simpletransformers.question_answering import QuestionAnsweringArgs, QuestionAnsweringModel

In [3]:
model_type = 'bert'

model_name = 'bert-base-cased'

if model_type == 'bert':
    model_name = 'bert-base-cased'

elif model_type == 'roberta':
    model_name = 'distilbert-base-cased'

elif model_type == 'distilroberta':
    model_type = 'roberta'
    model_name = 'distilroberta-base'

elif model_type == 'electra-base':
    model_type = 'electra'
    model_name = 'google/electra-base-discriminator'

elif model_type == 'electra-small':
    model_type = 'electra'
    model_name = 'google/electra-small-discriminator'

elif model_type == 'xlnet':
    model_name = 'xlnet-based-cased'

In [4]:
model_args = QuestionAnsweringArgs()

model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size = 3
model_args.num_train_epochs = 5

In [5]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f'outputs/{model_type}/best_model',
    "best_model_dir": f'outputs/{model_type}/best_model',
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc";
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config":{"output_hidden_states": True}
}

In [6]:
model = QuestionAnsweringModel(
    model_type, model_name, args = train_args, use_cuda=False
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

In [7]:
model.train_model(train_0, eval_data=test_0)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sahil2024 (use `wandb login --relogin` to force relogin)


Epoch 1 of 5:   0%|          | 0/5 [00:05<?, ?it/s]

In [ ]:
result, text = model.eval_model(test_0)

In [ ]:
result

In [ ]:
to_predict = [
    {
        "qas": [
            {
                "question": "Horn Cable Television and Universal TV are examples of what?",
                "id": "56e1c227cd28a01900c67b32"
            },
            {
                "question": "What type of Tv startions replay AL=jazeera and CNN?",
                "id": "56e1c227cd28a01900c67b33"
            },
            {
                "question": "Where does SBC TV broadcast from?",
                "id": "56e1c227cd28a01900c67b34"
            },
            {
                "question": "What is the commercial capital of Puntland?",
                "id": "56e1c227cd28a01900c67b35"
            },
            {
                "question": "Howmany governemnt run channels do Puntland and Somaliland regions have?",
                "id": "56e1c227cd28a01900c67b36"
            },
            {
                "question": "How many private television networks does Somalia have?",
                "id": "5a550d72134fea001a0e186a"
            },
            {
                "question": "What are two public television networks?",
                "id": "5a550d72134fea001a0e186b"
            },
            {
                "question": "What is the commercial capital of Somalilnand?",
                "id": "5a550d72134fea001a0e186c"
            },
            {
                "question": "How many government-run TV channels are there in Somalia?",
                "id": "5a550d72134fea001a0e186d"
            },
            {
                "question": "What TV station broadcasts ABC?",
                "id": "5a550d72134fea001a0e186e"
            }
            ],
            "context": "Additionally, Somalia has several private television networks, including Horn Cable Television and Universal TV. Two such TV stations re-broadcast Al-Jazeera and CNN. Eastern Television Network and SBC TV air from Bosaso, the commercial capital of Puntland. The Puntland and Somaliland regions also each have one government-run TV channel, Puntland TV and Radio and Somaliland National TV, respectively."

    },
]

In [ ]:
answer, probabilities = model.predict(to_predict)

In [ ]:
answer[0]['answer']